In [ ]:
!wget https://filedn.com/lqyeLp7CcY45K7LkYnSUQdX/map/centroides.csv
!wget https://filedn.com/lqyeLp7CcY45K7LkYnSUQdX/map/fileiras.csv
!wget https://filedn.com/lqyeLp7CcY45K7LkYnSUQdX/map/ortofoto.tif

--2025-07-31 22:53:51--  https://filedn.com/lqyeLp7CcY45K7LkYnSUQdX/map/centroides.csv
Resolving filedn.com (filedn.com)... 74.120.9.25
Connecting to filedn.com (filedn.com)|74.120.9.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6367 (6.2K) [text/csv]
Saving to: ‘centroides.csv’

centroides.csv      100%[===================>]   6.22K  --.-KB/s    in 0s      

2025-07-31 22:53:51 (75.5 MB/s) - ‘centroides.csv’ saved [6367/6367]

--2025-07-31 22:53:51--  https://filedn.com/lqyeLp7CcY45K7LkYnSUQdX/map/fileiras.csv
Resolving filedn.com (filedn.com)... 74.120.9.25
Connecting to filedn.com (filedn.com)|74.120.9.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 917 [text/csv]
Saving to: ‘fileiras.csv’

fileiras.csv        100%[===================>]     917  --.-KB/s    in 0s      

2025-07-31 22:53:51 (217 MB/s) - ‘fileiras.csv’ saved [917/917]

--2025-07-31 22:53:51--  https://filedn.com/lqyeLp7CcY45K7LkYnSUQdX/map/ortofoto.tif
Resolv

In [ ]:
import cv2
import csv
import numpy as np



def importar_fileiras_csv(caminho_arquivo):
    """
    Importa um CSV com colunas: Fileira, Slope (m), Intercept (b)
    Retorna uma lista de tuplas (m, b) para cada fileira.
    """
    linhas = []
    with open(caminho_arquivo, mode='r', newline='') as f:
        reader = csv.DictReader(f)
        for row in reader:
            m = float(row['Slope (m)'])
            b = float(row['Intercept (b)'])
            linhas.append((m, b))
    return linhas

def importar_centroides_csv(caminho_arquivo):
    """
    Importa um CSV com colunas: Fileira, Slope (m), Intercept (b)
    Retorna uma lista de tuplas (m, b) para cada fileira.
    """
    linhas = []
    with open(caminho_arquivo, mode='r', newline='') as f:
        reader = csv.DictReader(f)
        for row in reader:
            m = float(row['x'])
            b = float(row['y'])
            linhas.append((m, b))
    return linhas

def media (i1, i2):
  return (i1+i2)/2

def drawLines(lines, color):
  # 4. Desenha cada reta, calculando dois pontos de interseção com as bordas
  for m, b in lines:
      pts = []
      # interseção com x=0
      y0 = b
      if 0 <= y0 <= h:
          pts.append((0, int(y0)))
      # interseção com x=w-1
      y1 = m*(w-1) + b
      if 0 <= y1 <= h:
          pts.append((w-1, int(y1)))
      # interseção com y=0 (se m != 0)
      if m != 0:
          x0 = -b/m
          if 0 <= x0 <= w:
              pts.append((int(x0), 0))
      # interseção com y=h-1
      if m != 0:
          x1 = (h-1 - b)/m
          if 0 <= x1 <= w:
              pts.append((int(x1), h-1))
      # Se temos pelo menos dois pontos válidos, desenha o segmento
      if len(pts) >= 2:
          cv2.line(output, pts[0], pts[1], color=color, thickness=2)

image_bgr = cv2.imread('Task-of-2025-07-21T031534487Z-orthophoto (0.02).tif')
centros_globais = importar_centroides_csv('centroides.csv')
linhas = importar_fileiras_csv('fileiras.csv')

# 1. Carrega a imagem original (BGR)
h, w = image_bgr.shape[:2]

# 2. Supondo que você já tenha:
#    centros_globais: lista de (x,y)
#    linhas:       lista de (m, b) para y = m*x + b

output = image_bgr.copy()

# 3. Desenha os centroides
for x, y in centros_globais:
    cv2.circle(output, (int(x), int(y)), radius=5, color=(0,0,255), thickness=-1)


sorted_linhas = sorted(linhas, key=lambda x: x[1])
center_line = []
# 4. Desenha cada reta, calculando dois pontos de interseção com as bordas
for idx, (m, b) in enumerate(sorted_linhas, start=1):
    if (idx<len(linhas)-2):
      center_line.append((m,(media(b,sorted_linhas[idx][1]))))


drawLines(center_line, color=(255,0,0))
drawLines(linhas, color=(0,0,255))

# 5. Salva em disco
#cv2.imwrite('output.png', output)

In [ ]:
from shapely.geometry import LineString, Polygon

def linhas_para_segmentos(linhas, quadrilatero):
    """
    Converte uma lista de retas (m, b) em segmentos limitados ao interior do quadrilátero.

    linhas: lista de tuplas (m, b)
    quadrilatero: lista de 4 tuplas (x, y)
    retorna: lista de segmentos, cada um como ((x1, y1), (x2, y2))
    """
    poly = Polygon(quadrilatero)
    segmentos = []

    # Extensão horizontal para garantir retas longas
    min_x, max_x = poly.bounds[0], poly.bounds[2]
    padding = (max_x - min_x) * 10
    x_range = (min_x - padding, max_x + padding)

    for m, b in linhas:
        x1, x2 = x_range
        y1 = m * x1 + b
        y2 = m * x2 + b

        reta = LineString([(x1, y1), (x2, y2)])
        intersecao = reta.intersection(poly)

        if intersecao.is_empty:
            continue

        # Se a interseção é um segmento (LineString com 2 pontos)
        if intersecao.geom_type == 'LineString':
            coords = list(intersecao.coords)
            if len(coords) == 2:
                segmentos.append((tuple(coords[0]), tuple(coords[1])))

        # Se for uma coleção de segmentos
        elif intersecao.geom_type == 'MultiLineString':
            for seg in intersecao.geoms:
                coords = list(seg.coords)
                if len(coords) == 2:
                    segmentos.append((tuple(coords[0]), tuple(coords[1])))

        # Ignora outros casos (ponto, etc.)

    return segmentos


def drawQuadrilatero(output, quadrilatero, color=(255, 0, 0), thickness=2):
    """
    Desenha um quadrilátero na imagem `output`.

    - quadrilatero: lista de 4 tuplas (x, y)
    - output: imagem (numpy array)
    - color: cor BGR (padrão azul)
    - thickness: espessura da linha
    """
    pts = np.array(quadrilatero, dtype=np.int32).reshape((-1, 1, 2))
    cv2.polylines(output, [pts], isClosed=True, color=color, thickness=thickness)



def drawSegments(segments, output, color, thickness=2):
    """
    Desenha segmentos de reta no formato ((x1, y1), (x2, y2)) na imagem `output`.

    - segments: lista de segmentos [(pt1, pt2)], onde pt1 e pt2 são (x, y)
    - output: imagem OpenCV onde desenhar
    - color: cor BGR, ex: (0, 255, 0)
    - thickness: espessura da linha
    """
    for (x1, y1), (x2, y2) in segments:
        cv2.line(output, (int(x1), int(y1)), (int(x2), int(y2)), color=color, thickness=thickness)

# Quadrilátero em sentido horário
quad = [(1265, 1527), (3207, 939), (3635, 1611), (1666, 2200)]

segmentos = linhas_para_segmentos(center_line, quad)

print ("Segmentos encontrados:", len(segmentos))
drawSegments(segmentos, output, color=(0, 255,0), thickness=5)
drawQuadrilatero(output, quad, color=(255, 255, 0), thickness=5)

cv2.imwrite('output.png', output)



Segmentos encontrados: 5


True

In [ ]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 59.9 MB/s eta 0:00:00


In [ ]:
import rasterio
import numpy as np
from pyproj import Transformer

# Variáveis globais
_transform = None
_crs = None
_transformer = None
# Caminho para seu arquivo DSM
caminho_dsm = "Task-of-2025-07-14T185032197Z-dsm.tif"

# Abrindo o raster com rasterio
with rasterio.open(caminho_dsm) as src:
    dsm_array = src.read(1)  # Lê a primeira (e geralmente única) banda
    perfil = src.profile     # Metadados (resolução, CRS, etc.)
    transform = src.transform

def carregar_geoinfo(tif_path):
    """
    Carrega transformações geográficas do TIFF e armazena em variáveis globais.
    """
    global _transform, _crs, _transformer
    with rasterio.open(tif_path) as src:
        _transform = src.transform
        _crs = src.crs
        _transformer = Transformer.from_crs(_crs, "EPSG:4326", always_xy=True)


def pixel_para_latlon(col, row):
    """
    Converte coordenadas de pixel (col, row) em latitude e longitude usando globais.
    """
    if _transform is None or _transformer is None:
        raise ValueError("Você precisa chamar carregar_geoinfo(tif_path) antes.")

    x_geo, y_geo = _transform * (col, row)
    lon, lat = _transformer.transform(x_geo, y_geo)
    return lat, lon

carregar_geoinfo("Task-of-2025-07-21T031534487Z-orthophoto (0.02).tif")
coordenadas_finais = []

for (x1, y1), (x2, y2) in segmentos:
    lat1, lon1 = pixel_para_latlon(x1, y1)
    lat2, lon2 = pixel_para_latlon(x2, y2)
    alt1 = dsm_array[int(y1), int(x1)].item()
    alt2 = dsm_array[int(y2), int(x2)].item()
    if alt1== -9999.0:
      alt1=746
    if alt2== -9999.0:
      alt2=746

    coordenadas_finais.append(((lat1, lon1,alt1-730),(lat2, lon2,alt2-730)))

print(coordenadas_finais)

[((-23.057381524666123, -46.945002168792385, 16), (-23.057132387553025, -46.944046838870015, 16)), ((-23.057436163841974, -46.94496804416829, 16), (-23.05718649669044, -46.94401068216633, 16.54119873046875)), ((-23.05749103250176, -46.94493377619092, 16), (-23.057240833084762, -46.94397437357297, 16.09368896484375)), ((-23.057543810785695, -46.94490081371989, 16), (-23.05729309938055, -46.94393944822734, 15.71270751953125)), ((-23.057600873655865, -46.94486517529571, 16), (-23.057349608698054, -46.943901687578965, 14.93701171875))]


In [ ]:
def salvar_segmentos_em_kml(coordenadas_finais, nome_arquivo="segmentos.kml"):
    """
    Salva uma lista de segmentos ((lat1, lon1), (lat2, lon2)) em um arquivo KML.
    Cada segmento vira uma linha no Google Earth.
    """
    security_height=15
    with open(nome_arquivo, "w", encoding="utf-8") as f:
        f.write("""<?xml version="1.0" encoding="UTF-8"?>\n""")
        f.write("""<kml xmlns="http://www.opengis.net/kml/2.2">\n""")
        f.write("  <Document>\n")
        f.write(f"""    <Placemark>\n""")
        f.write(f"""      <name>Segmento 1 </name>\n""")
        f.write("""      <LineString>\n""")
        f.write("""        <tessellate>1</tessellate>\n""")
        f.write("""        <altitudeMode>relativeToGround</altitudeMode>\n""")
        f.write("""        <coordinates>\n""")
        for i, ((lat1, lon1,alt1), (lat2, lon2,alt2)) in enumerate(coordenadas_finais):
            if (i%2==0):
                f.write(f"""          {lon1},{lat1},{alt1+security_height} {lon1},{lat1},{alt1} {lon2},{lat2},{alt2} {lon2},{lat2},{alt2+security_height}\n""")
            else:
                f.write(f"""          {lon2},{lat2},{alt2+security_height} {lon2},{lat2},{alt2} {lon1},{lat1},{alt1} {lon1},{lat1},{alt1+security_height}\n""")

        f.write("""        </coordinates>\n""")
        f.write("""      </LineString>\n""")
        f.write("""    </Placemark>\n""")
        f.write("  </Document>\n")
        f.write("</kml>\n")

    print(f"KML salvo como '{nome_arquivo}'")

salvar_segmentos_em_kml(coordenadas_finais, "minha_saida.kml")

KML salvo como 'minha_saida.kml'


In [ ]:
import rasterio
import numpy as np

# Caminho para seu arquivo DSM
caminho_dsm = "Task-of-2025-07-14T185032197Z-dsm.tif"

# Abrindo o raster com rasterio
with rasterio.open(caminho_dsm) as src:
    dsm_array = src.read(1)  # Lê a primeira (e geralmente única) banda
    perfil = src.profile     # Metadados (resolução, CRS, etc.)
    transform = src.transform

# Exemplo: obter elevação de um ponto específico (linha, coluna)
linha, coluna = 4256,4256
elevacao = dsm_array[linha, coluna]
print(f"Elevação no ponto (linha={linha}, coluna={coluna}): {elevacao} metros")

# Exemplo: extrair elevação de todas as células
print("Estatísticas:")
print(f"Min: {np.min(dsm_array)}")
print(f"Max: {np.max(dsm_array)}")
print(f"Média: {np.mean(dsm_array)}")

Elevação no ponto (linha=4256, coluna=4256): 742.6900024414062 metros
Estatísticas:
Min: -9999.0
Max: 760.93798828125
Média: -3505.791259765625


In [ ]:
from lxml import etree

def criar_kml_rota(pontos, saida='rota.kml'):
    """
    Gera um arquivo KML compatível com o padrão DJI WPML a partir de uma lista de pontos.

    :param pontos: Lista de tuplas no formato (lat, lon, alt, heading)
    :param saida: Nome do arquivo de saída
    """
    NSMAP = {
        None: "http://www.opengis.net/kml/2.2",
        'wpml': "http://www.dji.com/wpmz/1.0.6"
    }

    def criar_placemark(idx, lat, lon, alt, heading):
        pm = etree.Element("Placemark")

        point = etree.SubElement(pm, "Point")
        coords = etree.SubElement(point, "coordinates")
        coords.text = f"{lon},{lat}"

        etree.SubElement(pm, "{http://www.dji.com/wpmz/1.0.6}index").text = str(idx)
        etree.SubElement(pm, "{http://www.dji.com/wpmz/1.0.6}executeHeight").text = str(alt)
        etree.SubElement(pm, "{http://www.dji.com/wpmz/1.0.6}waypointSpeed").text = "5"

        heading_param = etree.SubElement(pm, "{http://www.dji.com/wpmz/1.0.6}waypointHeadingParam")
        etree.SubElement(heading_param, "{http://www.dji.com/wpmz/1.0.6}waypointHeadingMode").text = "smoothTransition"
        etree.SubElement(heading_param, "{http://www.dji.com/wpmz/1.0.6}waypointHeadingAngle").text = str(heading)
        etree.SubElement(heading_param, "{http://www.dji.com/wpmz/1.0.6}waypointPoiPoint").text = "0.000000,0.000000,0.000000"
        etree.SubElement(heading_param, "{http://www.dji.com/wpmz/1.0.6}waypointHeadingAngleEnable").text = "1"
        etree.SubElement(heading_param, "{http://www.dji.com/wpmz/1.0.6}waypointHeadingPathMode").text = "followBadArc"
        etree.SubElement(heading_param, "{http://www.dji.com/wpmz/1.0.6}waypointHeadingPoiIndex").text = "0"

        turn_param = etree.SubElement(pm, "{http://www.dji.com/wpmz/1.0.6}waypointTurnParam")
        etree.SubElement(turn_param, "{http://www.dji.com/wpmz/1.0.6}waypointTurnMode").text = "toPointAndStopWithDiscontinuityCurvature"
        etree.SubElement(turn_param, "{http://www.dji.com/wpmz/1.0.6}waypointTurnDampingDist").text = "0"

        etree.SubElement(pm, "{http://www.dji.com/wpmz/1.0.6}useStraightLine").text = "1"

        gimbal_param = etree.SubElement(pm, "{http://www.dji.com/wpmz/1.0.6}waypointGimbalHeadingParam")
        etree.SubElement(gimbal_param, "{http://www.dji.com/wpmz/1.0.6}waypointGimbalPitchAngle").text = "0"
        etree.SubElement(gimbal_param, "{http://www.dji.com/wpmz/1.0.6}waypointGimbalYawAngle").text = "0"

        etree.SubElement(pm, "{http://www.dji.com/wpmz/1.0.6}isRisky").text = "0"
        etree.SubElement(pm, "{http://www.dji.com/wpmz/1.0.6}waypointWorkType").text = "0"

        return pm

    # Raiz do KML
    kml = etree.Element("kml", nsmap=NSMAP)
    doc = etree.SubElement(kml, "Document")

    # Configuração de missão
    mission_config = etree.SubElement(doc, "{http://www.dji.com/wpmz/1.0.6}missionConfig")
    etree.SubElement(mission_config, "{http://www.dji.com/wpmz/1.0.6}flyToWaylineMode").text = "pointToPoint"
    etree.SubElement(mission_config, "{http://www.dji.com/wpmz/1.0.6}finishAction").text = "goHome"
    etree.SubElement(mission_config, "{http://www.dji.com/wpmz/1.0.6}exitOnRCLost").text = "executeLostAction"
    etree.SubElement(mission_config, "{http://www.dji.com/wpmz/1.0.6}executeRCLostAction").text = "goBack"
    etree.SubElement(mission_config, "{http://www.dji.com/wpmz/1.0.6}takeOffSecurityHeight").text = "60"
    etree.SubElement(mission_config, "{http://www.dji.com/wpmz/1.0.6}globalTransitionalSpeed").text = "15"

    drone_info = etree.SubElement(mission_config, "{http://www.dji.com/wpmz/1.0.6}droneInfo")
    etree.SubElement(drone_info, "{http://www.dji.com/wpmz/1.0.6}droneEnumValue").text = "77"
    etree.SubElement(drone_info, "{http://www.dji.com/wpmz/1.0.6}droneSubEnumValue").text = "1"

    payload_info = etree.SubElement(mission_config, "{http://www.dji.com/wpmz/1.0.6}payloadInfo")
    etree.SubElement(payload_info, "{http://www.dji.com/wpmz/1.0.6}payloadEnumValue").text = "67"
    etree.SubElement(payload_info, "{http://www.dji.com/wpmz/1.0.6}payloadSubEnumValue").text = "0"
    etree.SubElement(payload_info, "{http://www.dji.com/wpmz/1.0.6}payloadPositionIndex").text = "0"

    # Folder com waypoints
    folder = etree.SubElement(doc, "Folder")
    etree.SubElement(folder, "{http://www.dji.com/wpmz/1.0.6}templateId").text = "0"
    etree.SubElement(folder, "{http://www.dji.com/wpmz/1.0.6}executeHeightMode").text = "relativeToStartPoint"
    etree.SubElement(folder, "{http://www.dji.com/wpmz/1.0.6}waylineId").text = "0"
    etree.SubElement(folder, "{http://www.dji.com/wpmz/1.0.6}distance").text = "0"
    etree.SubElement(folder, "{http://www.dji.com/wpmz/1.0.6}duration").text = "0"
    etree.SubElement(folder, "{http://www.dji.com/wpmz/1.0.6}autoFlightSpeed").text = "5"

    for idx, (lat, lon, alt, heading) in enumerate(pontos):
        folder.append(criar_placemark(idx, lat, lon, alt, heading))

    # Salvar o arquivo
    with open(saida, "wb") as f:
        f.write(etree.tostring(kml, pretty_print=True, xml_declaration=True, encoding="UTF-8"))

    print(f"KML salvo em: {saida}")


In [ ]:
from lxml import etree
import time

def gerar_kml_wpml_v2(pontos, saida='rota_wpml_v2.kml'):
    """
    Gera um arquivo KML no padrão DJI WPML com estrutura de missão waypoint.

    Parâmetros:
        pontos: lista de tuplas (lat, lon, alt, heading)
        saida: nome do arquivo KML de saída
    """
    NSMAP = {
        None: "http://www.opengis.net/kml/2.2",
        'wpml': "http://www.dji.com/wpmz/1.0.6"
    }

    def placemark(idx, lat, lon, alt, heading):
        pm = etree.Element("Placemark")
        point = etree.SubElement(pm, "Point")
        coords = etree.SubElement(point, "coordinates")
        coords.text = f"{lon},{lat}"

        etree.SubElement(pm, "{http://www.dji.com/wpmz/1.0.6}index").text = str(idx)
        etree.SubElement(pm, "{http://www.dji.com/wpmz/1.0.6}ellipsoidHeight").text = str(alt)
        etree.SubElement(pm, "{http://www.dji.com/wpmz/1.0.6}height").text = str(alt)

        heading_param = etree.SubElement(pm, "{http://www.dji.com/wpmz/1.0.6}waypointHeadingParam")
        etree.SubElement(heading_param, "{http://www.dji.com/wpmz/1.0.6}waypointHeadingMode").text = "smoothTransition"
        etree.SubElement(heading_param, "{http://www.dji.com/wpmz/1.0.6}waypointHeadingAngle").text = str(heading)
        etree.SubElement(heading_param, "{http://www.dji.com/wpmz/1.0.6}waypointPoiPoint").text = "0.000000,0.000000,0.000000"
        etree.SubElement(heading_param, "{http://www.dji.com/wpmz/1.0.6}waypointHeadingPathMode").text = "followBadArc"
        etree.SubElement(heading_param, "{http://www.dji.com/wpmz/1.0.6}waypointHeadingPoiIndex").text = "0"

        etree.SubElement(pm, "{http://www.dji.com/wpmz/1.0.6}useGlobalSpeed").text = "1"
        etree.SubElement(pm, "{http://www.dji.com/wpmz/1.0.6}useGlobalTurnParam").text = "1"
        etree.SubElement(pm, "{http://www.dji.com/wpmz/1.0.6}useStraightLine").text = "0"
        etree.SubElement(pm, "{http://www.dji.com/wpmz/1.0.6}isRisky").text = "0"

        return pm

    # Raiz do KML
    kml = etree.Element("kml", nsmap=NSMAP)
    doc = etree.SubElement(kml, "Document")

    now_ms = int(time.time() * 1000)
    etree.SubElement(doc, "{http://www.dji.com/wpmz/1.0.6}createTime").text = str(now_ms)
    etree.SubElement(doc, "{http://www.dji.com/wpmz/1.0.6}updateTime").text = str(now_ms + 100000)

    # Mission Config
    mission_config = etree.SubElement(doc, "{http://www.dji.com/wpmz/1.0.6}missionConfig")
    etree.SubElement(mission_config, "{http://www.dji.com/wpmz/1.0.6}flyToWaylineMode").text = "pointToPoint"
    etree.SubElement(mission_config, "{http://www.dji.com/wpmz/1.0.6}finishAction").text = "goHome"
    etree.SubElement(mission_config, "{http://www.dji.com/wpmz/1.0.6}exitOnRCLost").text = "executeLostAction"
    etree.SubElement(mission_config, "{http://www.dji.com/wpmz/1.0.6}executeRCLostAction").text = "goBack"
    etree.SubElement(mission_config, "{http://www.dji.com/wpmz/1.0.6}takeOffSecurityHeight").text = "60"
    etree.SubElement(mission_config, "{http://www.dji.com/wpmz/1.0.6}globalTransitionalSpeed").text = "15"

    drone_info = etree.SubElement(mission_config, "{http://www.dji.com/wpmz/1.0.6}droneInfo")
    etree.SubElement(drone_info, "{http://www.dji.com/wpmz/1.0.6}droneEnumValue").text = "77"
    etree.SubElement(drone_info, "{http://www.dji.com/wpmz/1.0.6}droneSubEnumValue").text = "1"

    payload_info = etree.SubElement(mission_config, "{http://www.dji.com/wpmz/1.0.6}payloadInfo")
    etree.SubElement(payload_info, "{http://www.dji.com/wpmz/1.0.6}payloadEnumValue").text = "67"
    etree.SubElement(payload_info, "{http://www.dji.com/wpmz/1.0.6}payloadSubEnumValue").text = "0"
    etree.SubElement(payload_info, "{http://www.dji.com/wpmz/1.0.6}payloadPositionIndex").text = "0"

    # Folder (Waypoints)
    folder = etree.SubElement(doc, "Folder")
    etree.SubElement(folder, "{http://www.dji.com/wpmz/1.0.6}templateType").text = "waypoint"
    etree.SubElement(folder, "{http://www.dji.com/wpmz/1.0.6}templateId").text = "0"

    coord_sys = etree.SubElement(folder, "{http://www.dji.com/wpmz/1.0.6}waylineCoordinateSysParam")
    etree.SubElement(coord_sys, "{http://www.dji.com/wpmz/1.0.6}coordinateMode").text = "WGS84"
    etree.SubElement(coord_sys, "{http://www.dji.com/wpmz/1.0.6}heightMode").text = "relativeToStartPoint"
    etree.SubElement(coord_sys, "{http://www.dji.com/wpmz/1.0.6}positioningType").text = "GPS"

    etree.SubElement(folder, "{http://www.dji.com/wpmz/1.0.6}autoFlightSpeed").text = "5"
    etree.SubElement(folder, "{http://www.dji.com/wpmz/1.0.6}globalHeight").text = "20"
    etree.SubElement(folder, "{http://www.dji.com/wpmz/1.0.6}caliFlightEnable").text = "0"
    etree.SubElement(folder, "{http://www.dji.com/wpmz/1.0.6}gimbalPitchMode").text = "manual"

    global_heading = etree.SubElement(folder, "{http://www.dji.com/wpmz/1.0.6}globalWaypointHeadingParam")
    etree.SubElement(global_heading, "{http://www.dji.com/wpmz/1.0.6}waypointHeadingMode").text = "manually"
    etree.SubElement(global_heading, "{http://www.dji.com/wpmz/1.0.6}waypointHeadingAngle").text = "0"
    etree.SubElement(global_heading, "{http://www.dji.com/wpmz/1.0.6}waypointPoiPoint").text = "0.000000,0.000000,0.000000"
    etree.SubElement(global_heading, "{http://www.dji.com/wpmz/1.0.6}waypointHeadingPoiIndex").text = "0"

    etree.SubElement(folder, "{http://www.dji.com/wpmz/1.0.6}globalWaypointTurnMode").text = "toPointAndStopWithDiscontinuityCurvature"
    etree.SubElement(folder, "{http://www.dji.com/wpmz/1.0.6}globalUseStraightLine").text = "1"

    for idx, (lat, lon, alt, heading) in enumerate(pontos):
        folder.append(placemark(idx, lat, lon, alt, heading))

    # PayloadParam
    payload_param = etree.SubElement(folder, "{http://www.dji.com/wpmz/1.0.6}payloadParam")
    etree.SubElement(payload_param, "{http://www.dji.com/wpmz/1.0.6}payloadPositionIndex").text = "0"
    etree.SubElement(payload_param, "{http://www.dji.com/wpmz/1.0.6}meteringMode").text = "average"
    etree.SubElement(payload_param, "{http://www.dji.com/wpmz/1.0.6}dewarpingEnable").text = "0"
    etree.SubElement(payload_param, "{http://www.dji.com/wpmz/1.0.6}returnMode").text = "singleReturnStrongest"
    etree.SubElement(payload_param, "{http://www.dji.com/wpmz/1.0.6}samplingRate").text = "240000"
    etree.SubElement(payload_param, "{http://www.dji.com/wpmz/1.0.6}scanningMode").text = "nonRepetitive"
    etree.SubElement(payload_param, "{http://www.dji.com/wpmz/1.0.6}modelColoringEnable").text = "0"

    # Escreve o arquivo
    with open(saida, "wb") as f:
        f.write(etree.tostring(kml, pretty_print=True, xml_declaration=True, encoding="UTF-8"))

    print(f"KML salvo em: {saida}")
